#                                           爬虫

### python库

 1、requests 用来获取页面内容
 2、beautiful soup

In [6]:
import time
import MySQLdb
import requests
from  bs4 import BeautifulSoup

In [12]:
url="https://bj.lianjia.com/zufang/"
responce=requests.get(url)
soup=BeautifulSoup(responce.text,'lxml')

In [15]:
links_div=soup.find_all('div',class_='pic-panel')
print(type(links_div))
print(links_div)
#links=[div.a.get('href') for div in links_div]

<class 'bs4.element.ResultSet'>
[<div class="pic-panel"><a href="https://bj.lianjia.com/zufang/101102483196.html" target="_blank"><img alt="东二环 7号线 冠城名敦道 西向开间" data-apart-layout="https://s1.ljcdn.com/feroot/pc/asset/img/new-version/default_block.png?_v=20180913164531875" data-img="https://image1.ljcdn.com/110000-inspection/24f9bfc5-fa7f-4472-825f-55036fcabd61.jpg.280x210.jpg" src="https://s1.ljcdn.com/feroot/pc/asset/img/new-version/default_block.png?_v=20180913164531875"/></a></div>, <div class="pic-panel"><a href="https://bj.lianjia.com/zufang/101102512882.html" target="_blank"><img alt="远洋沁山水 两居室 家具家电齐全自如装修风格" data-apart-layout="https://s1.ljcdn.com/feroot/pc/asset/img/new-version/default_block.png?_v=20180913164531875" data-img="https://image1.ljcdn.com/110000-inspection/rsp_pic_uploadff359236-c570-42bd-957d-972a138cb446.jpg.280x210.jpg" src="https://s1.ljcdn.com/feroot/pc/asset/img/new-version/default_block.png?_v=20180913164531875"/></a></div>, <div class="pic-panel"><a href="htt

In [14]:
links_div[1].a.get('href')

'https://bj.lianjia.com/zufang/101102512882.html'

In [6]:
def get_page(url):
    responce=requests.get(url)
    soup=BeautifulSoup(responce.text,'lxml')
    return soup

In [7]:
# 封装成函数，作用是获取列表页下面的所有租房页面的链接，返回一个链接列表
def get_links(link_url):
    soup=get_page(link_url)
    links_div=soup.find_all('div',class_='pic-panel')
    links=[div.a.get('href') for div in links_div]
    return links

In [44]:
def get_house_info(house_url):
   # house_url='https://bj.lianjia.com/zufang/101102691478.html'
    soup=get_page(house_url)
    price = soup.find('span',class_='total').text
    unit = soup.find('span',class_='unit').text.strip()
    house_info = soup.find_all('p')
    area=house_info[0].text[3:]
    area_n=house_info[0].text[:2]
    layout=house_info[1].text[5:]
    layout_n=house_info[1].text[:4]
    floor=house_info[2].text[3:]
    floor_n=house_info[2].text[:2]
    direction=house_info[3].text[5:]
    direction_n=house_info[3].text[:4]
    subway=house_info[4].text[3:]
    subway_n=house_info[4].text[:2]
    info={
        '价格':price,
        '单位':unit,
        area_n:area,
        layout_n:layout,
        floor_n:floor,
        direction_n:direction,
        subway_n:subway
    }
    return info

In [48]:
DATABASE = {
    'host':'127.0.0.1',
    'database':'Examination',
    'user':'root',
    'password':'123456',
    'charset':'utf8'
}
def get_db(setting):
    return MySQLdb.Connect(**setting)
def insert(db,house):
    values="'{}',"*6+"'{}'"
    sql_values=values.format(house['价格'],house['单位'],house['面积'],house['房屋户型'],house['楼层'],house['房屋朝向'],house['地铁'])
    sql="""insert into house(price,unit,area,layout,floor,direction,subway)
           values({})
    """.format(sql_values)
    try:
        print(sql)
        cursor=db.cursor()
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(e)

In [45]:
house_url='https://bj.lianjia.com/zufang/101102691478.html'
house_info=get_house_info(house_url)
house_info

{'价格': '8000',
 '单位': '元/月',
 '地铁': '暂无数据',
 '房屋户型': '2室1厅1卫  整租',
 '房屋朝向': '南',
 '楼层': '中楼层 (共24层)',
 '面积': '88平米 (套内70平米)'}

In [14]:
db=get_db(DATABASE)

In [43]:
insert(db,house_info)

In [49]:
db = get_db(DATABASE)
links = get_links("https://bj.lianjia.com/zufang/")
for link in links:
    time.sleep(2)
    print('获取一个房子信息成功')
    house=get_house_info(link)
    insert(db,house)

获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('6500','元/月','59平米 (套内59平米)','2室1厅1卫  整租','中楼层 (共6层)','南 北','距地铁10号线团结湖878米')
    
获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('13500','元/月','104平米 (套内81平米)','2室1厅2卫  整租','高楼层 (共9层)','南 北','距地铁10号线双井821米')
    
获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('6200','元/月','63平米 (套内44平米)','2室1厅1卫  ','中楼层 (共18层)','西南','距地铁10号线潘家园374米')
    
获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('8200','元/月','66平米 (套内46平米)','2室1厅1卫  整租','低楼层 (共12层)','南','距地铁2号线车公庄277米')
    
获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('9300','元/月','75平米 (套内50平米)','2室1厅1卫  ','高楼层 (共6层)','南 北','距地铁6号线北海北663米')
    
获取一个房子信息成功
insert into house(price,unit,area,layout,floor,direction,subway)
           values('6200','元/月','76平米 (套内67平米)

KeyboardInterrupt: 